## データの読み込み

mnistと呼ばれる有名な手書き文字（0~9）のデータセットを使用する。

In [22]:
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
    
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False, one_hot_label=True)
# 0~255 の値をスケーリングする
x_train, x_test = x_train / 255, x_test / 255

In [2]:
x_train.shape

(60000, 784)

In [3]:
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

# img_show(img)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(784,)
(28, 28)


## 2層のDNNを構築

In [4]:
import functions as F
import layers as L

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 重みとバイアスの初期化
        W1 = 0.01 * np.random.randn(I, H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H, O)
        b2 = np.zeros(O)
        
        # レイヤの生成
        self.layers = [
            L.Affine(W1, b1),
            L.Sigmoid(),
            L.Affine(W2, b2)
        ]
        
        self.loss_layer = L.SoftmaxWithLoss()
        
        # すべての重みと勾配をリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
        
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
        
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout


## 学習

In [23]:
import optimizer as opt

# ハイパーパラメータの設定
max_epoch = 10
batch_size = 5
hidden_size = 10
learning_rate = 1.0

# データの読み込み、モデルとオプティマイザの生成
x, t = x_train[:500], t_train[:500]

model = TwoLayerNet(input_size=784, hidden_size=hidden_size, output_size=10)
optimizer = opt.SGD(lr=learning_rate)

# 学習で使用する変数
data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0
loss_count = 0
loss_list = []

for epoch in range(max_epoch):
    # データのシャッフル
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]
    
    for iters in range(max_iters):
        batch_x = x[iters * batch_size : (iters + 1) * batch_size]
        batch_t = t[iters * batch_size : (iters + 1) * batch_size]
        
        # 勾配を求め、　パラメータを更新
        loss = model.forward(batch_x, batch_t)

        model.backward()
        optimizer.update(model.params, model.grads)
        
        total_loss += loss
        loss_count += 1
        
        # 定期的に学習経過を出力
        if (iters+1) % 100 == 0:
            avg_loss = total_loss / loss_count
            print('| epoch %d | iter %d / %d | loss %.2f' % (epoch + 1, iters + 1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0

| epoch 1 | iter 100 / 100 | loss 2.05
| epoch 2 | iter 100 / 100 | loss 1.24
| epoch 3 | iter 100 / 100 | loss 0.85
| epoch 4 | iter 100 / 100 | loss 0.62
| epoch 5 | iter 100 / 100 | loss 0.45
| epoch 6 | iter 100 / 100 | loss 0.37
| epoch 7 | iter 100 / 100 | loss 0.24
| epoch 8 | iter 100 / 100 | loss 0.20
| epoch 9 | iter 100 / 100 | loss 0.16
| epoch 10 | iter 100 / 100 | loss 0.11


## 精度の計算

In [26]:
pred_label = np.argmax(model.predict(x_test), axis=1)

In [27]:
true_label = np.argmax(t_test, axis=1)

In [30]:
np.sum(pred_label == true_label) / pred_label.shape[0]

0.7985

教師データ500枚に絞って2層のDNNでも**８０%弱**の精度になることが分かる。